In [2]:
import psycopg2

SCHEMA = 'historical'
CONN_DICT = {
    'host': 'main-us-e2.cmbsiiqeauby.us-east-2.rds.amazonaws.com',
    'user': 'doadmin',
    'password': 'i39kew8n7jcat7l9',
    'port': 5432,
    'dbname': 'main',
    'options': f'-c search_path={SCHEMA}'
}


def connect_to_db():
    return psycopg2.connect(**CONN_DICT)

def get_cur(conn):
    conn.rollback()
    return conn.cursor()

conn = connect_to_db()

In [3]:
from datetime import datetime, timedelta
t365 = datetime.now() - timedelta(days=365)
date_str = t365.strftime('%Y-%m-%d')

query = f"""
select match_id 
from historical_csgo.csgo_match_maps
where map_id not in (
	SELECT DISTINCT(map_id)
	FROM historical_csgo.csgo_map_rounds cmr 
	WHERE round_number != "t_score" + "ct_score"
	ORDER BY 1
) AND map_id NOT IN (
	SELECT DISTINCT(map_id) 
	FROM historical_csgo.csgo_map_rounds cmr
	GROUP BY 1
	HAVING max(round_number) < 16
	ORDER BY 1
) AND map_id NOT IN (
	SELECT distinct(map_id)
	FROM historical_csgo.csgo_match_maps cmm
	WHERE cmm.winner IS NULL OR cmm.winner = ''
) AND match_id not in (
	SELECT match_id
	FROM historical_csgo.csgo_match_data cmd
	WHERE cmd.date < '{date_str}'
)
"""
cur = get_cur(conn)
cur.execute(query)
match_ids = [x[0] for x in cur.fetchall()]
match_ids_str = ','.join([str(x) for x in match_ids])
print(f'Found {len(match_ids)} working. sample: {match_ids[:5]}')
cur.close()

Found 12052 working. sample: [0, 63934, 63934, 63934, 64820]


In [4]:
import pandas as pd
import numpy as np
import time

In [5]:
query = f"""
SELECT cpr.match_id,
       cpr.map_id,
       cmr.round_number,
       cpr.team_name,
       cpr.steam_id,
       MAX(won::INT)                                                                      AS won,
       MAX((cpr.team_name = cmr.t_team)::INT)                                             AS is_tside,
       MAX(cpr.round_start_money)                                                         AS parser_money,
       MAX(cmr.round_end_reason)                                                          AS round_end_reason,
       COUNT(crk.kill_id) FILTER ( WHERE cw.eq_class = 'AR')                              AS rifle_kills,
       COUNT(crk.kill_id) FILTER ( WHERE cw.eq_class = 'Sniper' AND crk.weapon_id != 309) AS sniper_kills, -- Remove AWP
       COUNT(crk.kill_id) FILTER ( WHERE cw.eq_class = 'Grenade')                         AS grenade_kills,
       COUNT(crk.kill_id) FILTER ( WHERE cw.eq_class = 'LMG')                             AS lmg_kills,
       COUNT(crk.kill_id) FILTER ( WHERE cw.eq_class = 'Shotgun')                         AS shotgun_kills,
       COUNT(crk.kill_id) FILTER ( WHERE cw.eq_class = 'SMG' AND crk.weapon_id != 106)    AS smg_kills,    --Remove P90
       COUNT(crk.kill_id) FILTER ( WHERE cw.eq_class = 'Pistol' AND crk.weapon_id != 8)   AS pistol_kills, --Remove CZ75
       COUNT(crk.kill_id) FILTER ( WHERE cw.eq_class = 'Melee' AND crk.weapon_id != 401)  AS melee_kills,  -- Remove Zeus
       COUNT(crk.kill_id) FILTER ( WHERE crk.weapon_id = 309)                             AS awp_kills,
       COUNT(crk.kill_id) FILTER ( WHERE crk.weapon_id = 106)                             AS p90_kills,
       COUNT(crk.kill_id) FILTER ( WHERE crk.weapon_id = 9)                               AS cz75_kills,
       COALESCE(MAX(bomb_events.bomb_plant), 0)                                           AS bomb_plants,
       COALESCE(MAX(bomb_events.bomb_defused), 0)                                         AS bomb_defused,
       COALESCE(MAX(bomb_events.bomb_exploded), 0)                                        AS bomb_exploded
FROM historical_csgo.csgo_player_rounds cpr
         LEFT JOIN historical_csgo.csgo_map_rounds cmr ON cmr.round_id = cpr.round_id
         LEFT JOIN historical_csgo.csgo_round_kills crk ON cpr.steam_id = crk.killer_steam_id AND cpr.round_id = crk.round_id
         LEFT JOIN historical_csgo.csgo_weapons cw ON cw.weapon_id = crk.weapon_id
         LEFT JOIN (
    SELECT crb.round_id,
           crb.steam_id,
           COUNT(crb.bomb_event_type) FILTER ( WHERE crb.bomb_event_type = 'BombPlanted')  AS bomb_plant,
           COUNT(crb.bomb_event_type) FILTER ( WHERE crb.bomb_event_type = 'BombDefused')  AS bomb_defused,
           COUNT(crb.bomb_event_type) FILTER ( WHERE crb.bomb_event_type = 'BombExploded') AS bomb_exploded
    FROM historical_csgo.csgo_rounds_be crb
    GROUP BY 1, 2
) AS bomb_events ON bomb_events.round_id = cpr.round_id AND bomb_events.steam_id = cpr.steam_id
WHERE
    cpr.match_id in ({match_ids_str})
GROUP BY 1, 2, 3, 4, 5;
"""
df = pd.read_sql(query, conn)
df

,match_id,map_id,round_number,team_name,steam_id,won,is_tside,parser_money,round_end_reason,rifle_kills,...,shotgun_kills,smg_kills,pistol_kills,melee_kills,awp_kills,p90_kills,cz75_kills,bomb_plants,bomb_defused,bomb_exploded
0,0,2,1,FURIA,76561197965021087,0,1,100,BombDefused,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,1,FURIA,76561197996370184,0,1,150,BombDefused,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,1,FURIA,76561198043769572,0,1,0,BombDefused,0,...,0,0,1,0,0,0,0,0,0,0
3,0,2,1,FURIA,76561198058500492,0,1,0,BombDefused,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,1,FURIA,76561198164970560,0,1,150,BombDefused,0,...,0,0,2,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073828,71357,71669,41,HAVU,76561197962710908,1,0,9950,CTWin,2,...,0,0,0,0,0,0,0,0,0,0
3073829,71357,71669,41,HAVU,76561197976197744,1,0,4650,CTWin,1,...,0,0,0,0,0,0,0,0,0,0
3073830,71357,71669,41,HAVU,76561198013172503,1,0,9650,CTWin,1,...,0,0,0,0,0,0,0,0,0,0
3073831,71357,71669,41,HAVU,76561198035334871,1,0,7100,CTWin,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
def streak(df, column_name, ref_value):
    df = df.sort_values(['map_id', 'steam_id', 'round_number'])
    prev_steam_id = df.iloc[0]['steam_id']

    df[column_name] = 0 
    prev_value = 0 # df.iloc[0][column_name]

    def func2(row):
        # non local variable ==> will use pre_value from the new_fun function
        nonlocal prev_value
        nonlocal prev_steam_id
        if prev_steam_id != row['steam_id']:
            # Since this is for larger dfs, we have to set the streak to one, because when we call apply we ommit the first row.
            # What if instead we just returned 0?
            prev_steam_id = row['steam_id']
            prev_value = 1
            return 1

        if row['won'] == ref_value:
            new_value =  prev_value + 1
        else:
            new_value = 0 

        prev_value = new_value
        return new_value

    # This line might throw a SettingWithCopyWarning warning
    df.iloc[:][column_name] = df.iloc[:].apply(func2, axis=1)
    return df

# This needs to be split changed because it currently takes the whole column and assigns the whole column it does not account for 
# multiple players in the DF
df = streak(df, 'win_streak', 1)
df = streak(df, 'loss_streak', 0)
df[['map_id', 'round_number', 'steam_id', 'won', 'is_tside', 'win_streak', 'loss_streak']].head(50)

,map_id,round_number,steam_id,won,is_tside,win_streak,loss_streak
5,2,1,76561197960710573,1,0,1,0
15,2,2,76561197960710573,1,0,2,0
25,2,3,76561197960710573,1,0,3,0
35,2,4,76561197960710573,1,0,4,0
45,2,5,76561197960710573,0,0,0,1
55,2,6,76561197960710573,0,0,0,2
65,2,7,76561197960710573,1,0,1,0
75,2,8,76561197960710573,1,0,2,0
85,2,9,76561197960710573,1,0,3,0
95,2,10,76561197960710573,0,0,0,1


In [7]:
ct_reasons = ['BombDefused', 'CTWin', 'TargetSaved']
t_reasons = ['TerroristsWin', 'TargetBombed']

df['new_money'] = 0
df['new_money'] = df['round_number'].apply(lambda x: 0 if x not in (1,16) else 800)
df

,match_id,map_id,round_number,team_name,steam_id,won,is_tside,parser_money,round_end_reason,rifle_kills,...,melee_kills,awp_kills,p90_kills,cz75_kills,bomb_plants,bomb_defused,bomb_exploded,win_streak,loss_streak,new_money
5,0,2,1,OG,76561197960710573,1,0,800,BombDefused,0,...,0,0,0,1,0,0,0,1,0,800
15,0,2,2,OG,76561197960710573,1,0,200,CTWin,2,...,0,0,0,0,0,0,0,2,0,0
25,0,2,3,OG,76561197960710573,1,0,1000,CTWin,0,...,0,0,0,0,0,0,0,3,0,0
35,0,2,4,OG,76561197960710573,1,0,2050,BombDefused,1,...,0,0,0,0,0,0,0,4,0,0
45,0,2,5,OG,76561197960710573,0,0,600,TargetBombed,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3072852,71354,71686,25,Young Ninjas,76561199132217533,1,0,11900,BombDefused,0,...,0,0,0,0,0,0,0,7,0,0
3072862,71354,71686,26,Young Ninjas,76561199132217533,0,0,14200,TerroristsWin,0,...,0,0,0,0,0,0,0,0,1,0
3072872,71354,71686,27,Young Ninjas,76561199132217533,0,0,9900,TargetBombed,0,...,0,0,0,0,0,0,0,0,2,0
3072882,71354,71686,28,Young Ninjas,76561199132217533,0,0,10700,TerroristsWin,2,...,0,0,0,0,0,0,0,0,3,0


In [8]:
# Need to activiate if a team wins two rounds, and then start counting loss bonuses

losing_bonus = {
    1: 1400,
    2: 1900,
    3: 2400,
    4: 2900,
    5: 3400
    # 5+ is 3400
}

after_2_wins_loss_bonus = {
    0: 1400,
    1: 1500,
    2: 2000,
    3: 2500,
    4: 3000
    # 4+ is 3000
}

BOMB_PLANT_BONUS = 300
PLAYER_BOMB_DEFUSE_BONUS = 300
TEAM_BOMB_DEFUSE_BONUS = 250
ROUND_WIN_BASE = 3250
# Round End conditions
# Enemy team elimnated: 3250
# CT Wins by time: 3250
# Bomb Defused: 3500
# Bomb Exploded: 3500

# bomb planted or defused = 300

In [9]:
gdf = df.groupby(['match_id', 'map_id', 'round_number']).max()[['bomb_defused', 'bomb_exploded']]
gdf = gdf.reset_index()
mdf = df.merge(gdf, on=['match_id', 'map_id', 'round_number'])
mdf['team_bomb_defused'] = mdf['bomb_defused_y']
del mdf['bomb_defused_y']
mdf['team_bomb_exploded'] = mdf['bomb_exploded_y']
del mdf['bomb_exploded_y']

mdf = mdf.sort_values(['steam_id', 'round_number'])
mdf['bomb_plant_bonus'] = mdf.apply(lambda x: BOMB_PLANT_BONUS if x['bomb_plants'] and x['is_tside'] else 0, axis=1) 
mdf['bomb_defuse_bonus'] = mdf.apply(lambda x: PLAYER_BOMB_DEFUSE_BONUS if x['bomb_defused_x'] and not x['is_tside'] else 0, axis=1) 
mdf['team_bomb_defuse_bonus'] = mdf.apply(lambda x: TEAM_BOMB_DEFUSE_BONUS if not x['is_tside'] and x['team_bomb_defused'] else 0, axis=1 )
mdf['team_bomb_explode_bonus'] = mdf.apply(lambda x: TEAM_BOMB_DEFUSE_BONUS if x['is_tside'] and x['team_bomb_exploded'] else 0, axis=1 )
mdf.head(25)

,match_id,map_id,round_number,team_name,steam_id,won,is_tside,parser_money,round_end_reason,rifle_kills,...,bomb_exploded_x,win_streak,loss_streak,new_money,team_bomb_defused,team_bomb_exploded,bomb_plant_bonus,bomb_defuse_bonus,team_bomb_defuse_bonus,team_bomb_explode_bonus
14209,63739,848,1,LDLC OL,76561197960282565,1,1,0,TerroristsWin,0,...,0,1,1,800,0,0,0,0,0,0
15409,64041,969,1,LDLC OL,76561197960282565,1,1,0,TerroristsWin,0,...,0,1,1,800,0,0,0,0,0,0
15579,64041,970,1,LDLC OL,76561197960282565,1,1,0,TerroristsWin,0,...,0,1,1,800,0,0,0,0,0,0
22859,63896,1460,1,LDLC,76561197960282565,1,1,4150,TerroristsWin,0,...,0,1,1,800,0,0,300,0,0,0
23049,63896,1461,1,LDLC,76561197960282565,1,1,0,TerroristsWin,0,...,0,1,1,800,0,0,0,0,0,0
23299,63896,1462,1,LDLC,76561197960282565,1,0,3700,CTWin,0,...,0,1,1,800,0,0,0,0,0,0
47286,64708,3711,1,LDLC OL,76561197960282565,1,0,0,CTWin,0,...,0,1,1,800,0,0,0,0,0,0
47496,64708,3712,1,LDLC OL,76561197960282565,1,1,0,TargetBombed,0,...,1,1,1,800,0,1,300,0,0,250
152244,64512,12297,1,LDLC OL,76561197960282565,0,1,0,CTWin,0,...,0,1,1,800,0,0,0,0,0,0
152474,64512,12298,1,LDLC OL,76561197960282565,1,1,0,TerroristsWin,0,...,0,1,1,800,0,0,0,0,0,0


In [10]:
def round_end_bonus(df):
    df = df.sort_values(['steam_id', 'round_number'])
    previous_row = df.iloc[0]
    previous_steam_id = df.iloc[0]['steam_id']

    latest_win_streak_after_loss = df.iloc[0]['win_streak']
    df['round_end_bonus'] = 0

    def inner(row):
        nonlocal previous_row
        nonlocal latest_win_streak_after_loss

        if row['steam_id'] != previous_steam_id:
            latest_win_streak_after_loss = 0 

        if row['won'] == 1:
            latest_win_streak_after_loss = 0 
            round_end_bonus = ROUND_WIN_BASE
        else:
            # Check if team has won at least two consecutive rounds
            if previous_row['win_streak'] >= 2 or latest_win_streak_after_loss >= 2:
                if latest_win_streak_after_loss <= previous_row['win_streak']:
                    latest_win_streak_after_loss = previous_row['win_streak'] 

                if latest_win_streak_after_loss >= 4 or row['loss_streak'] >= 4:
                    round_end_bonus = after_2_wins_loss_bonus[4]
                else:
                    round_end_bonus = after_2_wins_loss_bonus[row['loss_streak']]
            else:
                if row['loss_streak'] >= 5:
                    round_end_bonus = losing_bonus[5] 
                else:
                    round_end_bonus = losing_bonus[row['loss_streak']]

        row['round_end_bonus'] = round_end_bonus
        previous_row = row.copy()
        return row

    df = df.apply(inner, axis=1)
    return df

mdf = round_end_bonus(mdf)
    


In [11]:
mdf[['round_number', 'steam_id', 'won', 'is_tside', 'win_streak', 'loss_streak', 'round_end_bonus']].head(25)

,round_number,steam_id,won,is_tside,win_streak,loss_streak,round_end_bonus
14209,1,76561197960282565,1,1,1,1,3250
15409,1,76561197960282565,1,1,1,1,3250
15579,1,76561197960282565,1,1,1,1,3250
22859,1,76561197960282565,1,1,1,1,3250
23049,1,76561197960282565,1,1,1,1,3250
23299,1,76561197960282565,1,0,1,1,3250
47286,1,76561197960282565,1,0,1,1,3250
47496,1,76561197960282565,1,1,1,1,3250
152244,1,76561197960282565,0,1,1,1,1400
152474,1,76561197960282565,1,1,1,1,3250


In [12]:
weapon_money = {
    'Pistol': 300,
    'Knife': 1500,
    'Cz75': 100,
    'SMG': 600,
    'P90': 300,
    'Shotgun': 900,
    'Rifle': 300,
    'Sniper': 300,
    'Machine Gun': 300,
    'AWP': 100,
    'Grenade': 300,
    'Zeus': 0
}
mdf['ar_bonus'] = mdf['rifle_kills'].apply(lambda x: x * weapon_money['Rifle']) 
mdf['sniper_bonus'] = mdf['sniper_kills'].apply(lambda x: x*weapon_money['Sniper'])
mdf['grenade_bonus'] = mdf['grenade_kills'].apply(lambda x: x*weapon_money['Grenade'])
mdf['lmg_bonus'] = mdf['lmg_kills'].apply(lambda x: x*weapon_money['Machine Gun'])
mdf['shotgun_bonus'] = mdf['shotgun_kills'].apply(lambda x: x*weapon_money['Shotgun'])
mdf['smg_bonus'] = mdf['smg_kills'].apply(lambda x: x*weapon_money['SMG'])
mdf['pistol_bonus'] = mdf['pistol_kills'].apply(lambda x: x*weapon_money['Pistol'])
mdf['melee_bonus'] = mdf['melee_kills'].apply(lambda x: x*weapon_money['Knife']) 
mdf['awp_bonus'] = mdf['awp_kills'].apply(lambda x: x*weapon_money['AWP'])
mdf['p90_bonus'] = mdf['p90_kills'].apply(lambda x: x*weapon_money['P90'])
mdf['cz75_bonus'] = mdf['cz75_kills'].apply(lambda x: x*weapon_money['Cz75'])

In [13]:
mdf['total_money_made'] = mdf[[x for x in mdf.columns if x.endswith('_bonus')]].sum(axis=1)
mdf[['match_id', 'map_id', 'round_number', 'steam_id', 'total_money_made', 'parser_money']]

,match_id,map_id,round_number,steam_id,total_money_made,parser_money
14209,63739,848,1,76561197960282565,6250,0
15409,64041,969,1,76561197960282565,3950,0
15579,64041,970,1,76561197960282565,3250,0
22859,63896,1460,1,76561197960282565,4150,4150
23049,63896,1461,1,76561197960282565,3250,0
...,...,...,...,...,...,...
3018134,71244,71466,37,76561199241953370,3650,3650
3018144,71244,71466,38,76561199241953370,1500,5000
3018154,71244,71466,39,76561199241953370,2200,1050
3018164,71244,71466,40,76561199241953370,3550,5200


In [14]:
mdf[mdf['map_id']==14908][[x for x in mdf.columns if x.endswith('_bonus')]]

,bomb_plant_bonus,bomb_defuse_bonus,team_bomb_defuse_bonus,team_bomb_explode_bonus,round_end_bonus,ar_bonus,sniper_bonus,grenade_bonus,lmg_bonus,shotgun_bonus,smg_bonus,pistol_bonus,melee_bonus,awp_bonus,p90_bonus,cz75_bonus


In [15]:
money_df = mdf.groupby(['match_id', 'map_id', 'steam_id']).sum()['total_money_made']
money_df = money_df.reset_index()
money_df

,match_id,map_id,steam_id,total_money_made
0,0,2,76561197960710573,69000
1,0,2,76561197965021087,61900
2,0,2,76561197996370184,58700
3,0,2,76561198013243326,70500
4,0,2,76561198016432560,70400
...,...,...,...,...
121342,71357,71669,76561198035334871,122100
121343,71357,71669,76561198065448360,115850
121344,71357,71669,76561198186878873,122550
121345,71357,71669,76561198258187741,123300


In [27]:
match_money_df = money_df.groupby(['match_id', 'steam_id']).agg(total_money_per_map=('total_money_made', 'mean'), total_money=('total_money_made','sum')).reset_index()
match_money_df.to_csv('csgo_money.csv',index=False)
print('Shape', match_money_df.shape)
match_money_df.head()

Shape (58631, 4)


,match_id,steam_id,total_money_per_map,total_money
0,0,76561197960710573,69000.0,69000
1,0,76561197965021087,61900.0,61900
2,0,76561197996370184,58700.0,58700
3,0,76561198013243326,70500.0,70500
4,0,76561198016432560,70400.0,70400


In [18]:
money_df.shapekkk

(121347, 4)